In [1]:
%cd yolov5

D:\MyData\yolov5_data\yolov5


# For Yolov5

In [3]:
import os
import shutil
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import letterbox
from utils.general import (non_max_suppression, scale_coords, strip_optimizer, check_img_size,xyxy2xywh)
from utils.torch_utils import select_device
import numpy as np


from pystackreg import StackReg

from uuid import uuid4
import requests

import shutil



#  For sahi

In [4]:
# import required functions, classes
from sahi.model import Yolov5DetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from IPython.display import Image
import cv2


#  Functions Used

In [5]:
# load model
def init():
    global model
    global names
    global imgsz
    global device
    imgsz = 640
    device = select_device("")
    # Load model
    model = attempt_load("runs/train/exp12/weights/best.pt", map_location=device)  # load FP32 model
    imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size


    names = model.module.names if hasattr(model, 'module') else model.names

# get fdetections
def detect_image(img_main):
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)

    img = letterbox(img_main, new_shape=imgsz)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device)

    img = img.float()  # uint8 to fp16/32
    img /= 255.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # pred
    pred = model(img, augment=True)[0]

    # Apply NMS
    pred = non_max_suppression(pred, 0.4, 0.2, classes=None, agnostic=False)


    results = []
    # Process detections
    for i, det in enumerate(pred):  # detections per image
        if det is not None and len(det):
            # Rescale boxes from img_size to img_main size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img_main.shape).round()
            
            
            # Write results
            for *xyxy, conf, cls in reversed(det):
                results.append({
                    'xmin' : int(xyxy[0]), 
                    'ymin' : int(xyxy[1]),
                    'xmax' : int(xyxy[2]), 
                    'ymax' : int(xyxy[3]),
                    'class' : names[int(cls)]
                })
    return results

# get b_boxes
def get_b_boxes(img_main):
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)

    img = letterbox(img_main, new_shape=imgsz)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device)

    img = img.float()  # uint8 to fp16/32
    img /= 255.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # pred
    pred = model(img, augment=True)[0]

    # Apply NMS
    pred = non_max_suppression(pred, 0.3, 0.3, classes=None, agnostic=False)


    Boxes = []
    # Process detections
    for i, det in enumerate(pred):  # detections per image
        if det is not None and len(det):
            # Rescale boxes from img_size to img_main size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img_main.shape).round()
            
            
            # Write results
            for *xyxy, conf, cls in reversed(det):
                Boxes.append([int(xyxy[0]), int(xyxy[1]),int(xyxy[2]),int(xyxy[3])])
    return Boxes

def compute_iou(bounding_box_1, bounding_box_2):
    """
    This function computes the Intersection over Union of two bounding boxes.
    :param bounding_box_1: bounding box coordinates of 1st box
    :param bounding_box_2:
    :return: computed IoU
    """
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(bounding_box_1[0], bounding_box_2[0])
    yA = max(bounding_box_1[1], bounding_box_2[1])
    xB = min(bounding_box_1[2], bounding_box_2[2])
    yB = min(bounding_box_1[3], bounding_box_2[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-trut rectangles
    boxAArea = (bounding_box_1[2] - bounding_box_1[0] + 1) * (bounding_box_1[3] - bounding_box_1[1] + 1)
    boxBArea = (bounding_box_2[2] - bounding_box_2[0] + 1) * (bounding_box_2[3] - bounding_box_2[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

def resize_with_padding(img,new_image_width,new_image_height):
    old_image_height, old_image_width, channels = img.shape

    # create new image of desired size and color (blue) for padding
    color = (0,0,0)
    
    result = np.full((new_image_height,new_image_width, channels), color, dtype=np.uint8)

    # compute center offset
    x_center = (new_image_width - old_image_width) // 2
    y_center = (new_image_height - old_image_height) // 2

    # copy img image into center of result image
    result[y_center:y_center+old_image_height, 
           x_center:x_center+old_image_width] = img
    
    return result



sr = StackReg(StackReg.RIGID_BODY)
class ImageRegister():
    def __init__(self):
        pass

    def register_images(self, base_image, images_list):
        """ Register images using pystackreg """
        
        # Read base image, get shape and remove from directory
        base_image = cv2.imread(base_image)
        base_shape = base_image.shape
        

        for image_path in images_list: # Iterate over images (ref) and register each image againt base image

            # read ref image and resize to base image size
            mov_image = cv2.resize(cv2.imread(image_path, -1), (base_shape[1], base_shape[0]))

            # Register image
            sr.register(base_image[:,:,0], mov_image[:,:,0])
            out_aff_0 = sr.transform(mov_image[:, :, 0])
            out_aff_1 = sr.transform(mov_image[:,:,1])
            out_aff_2 = sr.transform(mov_image[:,:,2])

            ref_shape=mov_image.shape
            out_aff = np.zeros(ref_shape)
            out_aff[:,:,0] = out_aff_0
            out_aff[:,:,1] = out_aff_1
            out_aff[:,:,2] = out_aff_2

            #stacked_image = np.hstack((base_image, cv2.normalize(src=out_aff, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)))
            
            
            # save registered image to disk
            ##stacked_image_name = f"{uuid4()}.jpg"           
            #cv2.imwrite("Testing_Changes/1C.jpg", cv2.normalize(src=out_aff, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U))
        
        return cv2.normalize(src=out_aff, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)


#  Loading Model for Detetection by Slicing ( Sahi )

In [6]:
yolov5_model_path = 'D:/MyData/yolov5_data/yolov5/runs/train/exp12/weights/best.pt'
detection_model = Yolov5DetectionModel(
    model_path=yolov5_model_path ,
    confidence_threshold=0.3,
    device="cuda:0", 
    image_size=640
    #load_at_init=False
)


                 from  n    params  module                                  arguments                     
  0                -1  1      8800  yolov5.models.common.Conv               [3, 80, 6, 2, 2]              
  1                -1  1    115520  yolov5.models.common.Conv               [80, 160, 3, 2]               
  2                -1  4    309120  yolov5.models.common.C3                 [160, 160, 4]                 
  3                -1  1    461440  yolov5.models.common.Conv               [160, 320, 3, 2]              
  4                -1  8   2259200  yolov5.models.common.C3                 [320, 320, 8]                 
  5                -1  1   1844480  yolov5.models.common.Conv               [320, 640, 3, 2]              
  6                -1 12  13125120  yolov5.models.common.C3                 [640, 640, 12]                
  7                -1  1   7375360  yolov5.models.common.Conv               [640, 1280, 3, 2]             
  8                -1  4  19676160  

# Change Detection By Detections Using Slicing 

In [ ]:
# for Checking changes in Building
import cv2
import matplotlib.pyplot as plt

# load Model


A='D:\MyData\yolov5_data\sahi\Test2_A.jpg'
B='D:\MyData\yolov5_data\sahi\Test2_B.jpg'
C='D:\MyData\yolov5_data\sahi\Test2_C.jpg'


# read image using opnecv
After = cv2.imread(A)

# registering before image using after as a refrence
IR=ImageRegister()
Before=IR.register_images(A,[B])
cv2.imwrite(B,Before)

#Before = cv2.imread("Testing_Changes/4B.jpg")

Results_A=get_sliced_prediction(
    A,
    detection_model,
    slice_height =640,
    slice_width = 640,
    overlap_height_ratio = 0.2,
    overlap_width_ratio = 0.2,
     perform_standard_pred=False,
     postprocess_type="NMS",
    postprocess_match_metric="IOS",
    postprocess_match_threshold=0.4
)

Boxes_A = [] # pass image 
for prediction in Results_A.object_prediction_list:
    Boxes_A.append(prediction.bbox.to_voc_bbox())

for box in Boxes_A:
    cv2.rectangle(After, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2) # draw rectangle



Results_B=get_sliced_prediction(
    B,
    detection_model,
    slice_height =640,
    slice_width = 640,
    overlap_height_ratio = 0.2,
    overlap_width_ratio = 0.2,
     perform_standard_pred=False,
     postprocess_type="NMS",
    postprocess_match_metric="IOS",
    postprocess_match_threshold=0.4
)

Boxes_B = [] # pass image
for prediction in Results_B.object_prediction_list:
    Boxes_B.append(prediction.bbox.to_voc_bbox())

for box in Boxes_B:
    cv2.rectangle(Before, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2) # draw rectangle



## Detecting Changes

Change = cv2.imread(A)
cv2.imwrite('D:\MyData\yolov5_data\sahi\Test2_A_R.jpg',After)
cv2.imwrite('D:\MyData\yolov5_data\sahi\Test2_B_R.jpg',Before)

for box in Boxes_A:
    c=0
    for box1 in Boxes_B:
        if(compute_iou(box,box1))<0.4:
            c+=1
    if c==len(Boxes_B):
        cv2.rectangle(Change, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2) # draw rectangle
        cv2.putText(Change, 'Changed', (box[0],box[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 5)
        
     
cv2.imwrite(C,Change)